In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from cmdstanpy import CmdStanModel

Below we repeat the analysis that was done in the original simulation.

In [ ]:
chicks = pd.read_table("chickens.dat", sep="\\s+")
chicks.head()

,freq,sham_n,sham_est,sham_se,exposed_n,exposed_est,exposed_se
0,1,32,0.995,0.041,32,1.036,0.041
1,15,32,1.013,0.042,36,1.173,0.034
2,30,32,1.033,0.032,32,1.107,0.035
3,45,32,0.990,0.032,32,1.181,0.052
4,60,32,0.998,0.040,32,1.136,0.044


In [ ]:
chick_model = CmdStanModel(stan_file="dynamic_hier.stan")
chick_data = {
    "num_expts": len(chicks),
    "avg_treated_response": chicks["exposed_est"] - 1,
    "avg_control_response": chicks["sham_est"] - 1,
    "treated_se": chicks["exposed_se"],
    "control_se": chicks["sham_se"],
    "expt_id": list(range(1, len(chicks) + 1)),
}
chick_fit = chick_model.sample(data=chick_data, adapt_delta=0.9)

17:29:23 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

17:29:24 - cmdstanpy - INFO - CmdStan done processing.


The function below simulates a dataset for `num_experiments` experiments, all assumed to have the same proportion `prop_treatment` allocated to the treatment group. Here one complication is that the original simulation assumes all experiments have the same number of subjects in the control and treatment groups.

The original assumptions are:

$$
\begin{align*}
    y_{j1} \mid \theta_j, b_j &\sim N(\theta_j + b_j, s_{j1}) \\
    y_{j0} \mid b_j &\sim N(b_j, s_{j0}) \\
    \theta_j &\sim N(\mu_\theta, \sigma_\theta) \\
    b_j &\sim N(\mu_b, \sigma_b)
\end{align*}
$$

These assumptions mostly still work here since $s_{j1}$ and $s_{j0}$ need not be the same for all experiments $j$. However, in the original simulation, it was assumed that $s_{j1} = s_{j0} = 0.04$ for all $j$. This is reasonable if the treatment and control groups have the same size and if we expect the treatment/sham effects to have similar spread, which was empirically shown for the chicken dataset. 

However, if the groups have unequal numbers of subjects, then we expect the group with fewer subjects to have higher variance. We let $Y_{j0}^{(i)}$ and $Y_{j1}^{(k)}$ be the outcomes for the $i$-th and $k$-th individuals in the control and treatment groups respectively for experiment $j$. Let the $j$-th experiment have $N_{j0}$ and $N_{j1}$ subjects in control and treatment groups respectively. If the number of subjects in each group is at least 30, then assuming the individual outcomes are iid, the CLT gives

$$ y_{j1} = \frac{1}{N_{j1}} \sum_{k=1}^{N_{j1}} Y_{j1}^{(k)} \rightarrow N(\mu_{j1}, \sigma_{j1}) $$

where $\sigma_{j1} = \sqrt{\frac{\text{var}(Y_{j1})}{N_{j1}}}$ and similarly for $y_{j0}$. We identify $\mu_{j1}$ with $\theta_j + b_j$ and $\sigma_{j1}$ with $s_{j1}$. Since $s_{j1} = 0.04$ in the original simulation, we use $\text{var}(Y_{j1}) = 32 \times (0.04)^2$ for all experiments $j$; likewise for $\text{var}(Y_{j0})$.

*Thought: What if the sample sizes are too small for the CLT? Maybe bootstrap or make distributional assumptions on $Y_{j1}^{(k)}$?*

In [52]:
def fake_expts_with_prop(
    num_subjects_per_expt,
    prop_treatment,
    mu_b,
    mu_theta,
    sigma_b,
    sigma_theta,
    sigma_treatment,
    sigma_control,
):
    """
    Generate synthetic data with a specified proportion of treated subjects.

    Args:
    num_subjects_per_expt: A list of integers, the number of subjects in each
        experiment.
    prop_treatment: A float, the proportion of subjects that are treated.
    mu_b: A float, the true (mean) of the control group response.
    mu_theta: A float, the true (mean) treatment effect.
    sigma_b: A float, the standard deviation of b_j.
    sigma_theta: A float, the standard deviation of theta_j.
    sigma_treatment: A float, the standard deviation of the treated group
        response.
    sigma_control: A float, the standard deviation of the control group
        response.
    """
    assert len(num_subjects_per_expt) == len(prop_treatment)
    assert sigma_theta >= 0 and sigma_b >= 0
    assert sigma_treatment >= 0 and sigma_control >= 0

    num_expts = len(num_subjects_per_expt)
    num_treated = np.floor(prop_treatment * num_subjects_per_expt).astype(int)
    num_control = num_subjects_per_expt - num_treated

    sigma_y1 = sigma_treatment / np.sqrt(num_treated)
    sigma_y0 = sigma_control / np.sqrt(num_control)

    theta = np.random.normal(mu_theta, sigma_theta, num_expts)
    b = np.random.normal(mu_b, sigma_b, num_expts)

    return {
        "num_expts": len(num_subjects_per_expt),
        "avg_treated_response": np.random.normal(theta + b, sigma_y1),
        "avg_control_response": np.random.normal(b, sigma_y0),
        "treated_se": sigma_treatment / np.sqrt(num_treated),
        "control_se": sigma_control / np.sqrt(num_control),
        "expt_id": list(range(1, num_expts + 1)),
    }

In [55]:
proportions = np.array([0.5, 0.5, 0.5, 0.3, 0.2])
num_subjects_per_expt = np.array([30, 30, 30, 30, 30])
sigma_y1 = (32 ** 0.5) * 0.04 # see above for explanation
sigma_y0 = (32 ** 0.5) * 0.04
synthetic_data = fake_expts_with_prop(
    num_subjects_per_expt,
    proportions,
    mu_b=0,
    mu_theta=1,
    sigma_b=1,
    sigma_theta=1,
    sigma_treatment=sigma_y1,
    sigma_control=sigma_y0,
)

print(synthetic_data)

{'num_expts': 5, 'avg_treated_response': array([ 3.18610929,  0.0279549 ,  1.77932519, -1.35866756,  1.37670522]), 'avg_control_response': array([ 0.85754649,  0.02449063,  0.23535277, -1.37313463,  0.95365174]), 'treated_se': array([0.05842374, 0.05842374, 0.05842374, 0.07542472, 0.09237604]), 'control_se': array([0.05842374, 0.05842374, 0.05842374, 0.04937707, 0.04618802]), 'expt_id': [1, 2, 3, 4, 5]}
